# Programming Assignment: Линейная регрессия: прогноз оклада по описанию вакансии
# Linear Regression: Salary forecast by job description

## Ridge 
### tf-idf 
### one hot encoding

In [1]:
import numpy as np
import pandas

1. Загрузите данные об описаниях вакансий и соответствующих годовых зарплатах из файла salary-train.csv (либо его заархивированную версию salary-train.zip).

In [2]:
data=pandas.read_csv('salary-train.csv')
data.info()
print '-'*20
print data.describe()
print '-'*20
print data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60000 entries, 0 to 59999
Data columns (total 4 columns):
FullDescription       60000 non-null object
LocationNormalized    60000 non-null object
ContractTime          44418 non-null object
SalaryNormalized      60000 non-null int64
dtypes: int64(1), object(3)
memory usage: 1.8+ MB
--------------------
       SalaryNormalized
count      60000.000000
mean       34084.931500
std        17593.539646
min         5000.000000
25%        21500.000000
50%        30000.000000
75%        42500.000000
max       200000.000000
--------------------
                                     FullDescription LocationNormalized  \
0  International Sales Manager London ****k  ****...             London   
1  An ideal opportunity for an individual that ha...             London   
2  Online Content and Brand Manager// Luxury Reta...  South East London   
3  A great local marketleader is seeking a perman...            Dereham   
4  Registered Nurse / RGN  Nursing

2- Проведите предобработку:

- Приведите тексты к нижнему регистру (text.lower()).
- Замените все, кроме букв и цифр, на пробелы — это облегчит дальнейшее разделение текста на слова. Для такой замены в строке text подходит следующий вызов: re.sub('[^a-zA-Z0-9]', ' ', text). Также можно воспользоваться методом replace у DataFrame, чтобы сразу преобразовать все тексты:

train['FullDescription'] = train['FullDescription'].replace('[^a-zA-Z0-9]', ' ', regex = True)

- Замените пропуски в столбцах LocationNormalized и ContractTime на специальную строку 'nan'. Код для этого был приведен выше.

2 - Guide preprocessing:

- Bring texts to lowercase (text.lower()).
- Replace everything except letters and numbers with spaces — it will facilitate the further division of the text into words. For such replacement in the string text fits the following call: re.sub('[^a-zA-Z0-9]', ' ', text). You can also use the replace method from DataFrame to convert all the texts:

train['FullDescription'] = train['FullDescription'].replace('[^a-zA-Z0-9]', ' ', regex = True)

- Replace the blanks in the columns LocationNormalized and ContractTime on the special string 'nan'. The code for this was given above.


In [3]:
data['LocationNormalized'].fillna('nan', inplace=True)
data['ContractTime'].fillna('nan', inplace=True)
data['FullDescription'] = data['FullDescription'].str.lower().replace('[^a-zA-Z0-9]', ' ', regex = True)
y=data['SalaryNormalized']

## Tf-idf 

- Примените TfidfVectorizer для преобразования текстов в векторы признаков. Оставьте только те слова, которые встречаются хотя бы в 5 объектах (параметр min_df у TfidfVectorizer).

- Use TfidfVectorizer to transform texts into vectors of characteristics. Leave only those words that occur at least 5 objects (parameter min_df have TfidfVectorizer).

<img src="https://deeplearning4j.org/img/tfidf.png"/>
<hr>
<img src="http://www.joyofdata.de/blog/wp-content/uploads/2014/02/tf-idf.png"/>

In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer=TfidfVectorizer(min_df=5)
corpus_tf_idf = vectorizer.fit_transform(data['FullDescription'])

## One hot encoding

- Примените DictVectorizer для получения one-hot-кодирования признаков LocationNormalized и ContractTime.
- Use DictVectorizer to obtain one-hot encoded signs LocationNormalized and ContractTime.

In [5]:
from sklearn.feature_extraction import DictVectorizer
enc = DictVectorizer()
Xnew =enc.fit_transform(data[['LocationNormalized', 'ContractTime']].to_dict('records'))

- Объедините все полученные признаки в одну матрицу "объекты-признаки". Обратите внимание, что матрицы для текстов и категориальных признаков являются разреженными. Для объединения их столбцов нужно воспользоваться функцией scipy.sparse.hstack.
- Combine all the signs into one matrix "objects-features". Note that the matrix for text and categorical features are sparse. To merge columns you need to use the function scipy.sparse.hstack.

In [6]:
from scipy.sparse import coo_matrix, hstack
X=hstack([Xnew,corpus_tf_idf])

3- Обучите гребневую регрессию с параметрами alpha=1 и random_state=241. Целевая переменная записана в столбце SalaryNormalized.

3- Train the ridge regression with parameters alpha=1, random_state=241. The target variable is written in the column SalaryNormalized.

In [7]:
from sklearn.linear_model import Ridge
import numpy as np
clf = Ridge(alpha=1.0,random_state=241)
clf.fit(X, y) 

Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=241, solver='auto', tol=0.001)

4- Постройте прогнозы для двух примеров из файла salary-test-mini.csv. Значения полученных прогнозов являются ответом на задание. Укажите их через пробел.

4- Build forecasts for the two examples from the file salary-test-mini.csv. The values obtained predictions are a response to the task. Specify them separated by space.

In [8]:
test=pandas.read_csv('test1.csv')
test['LocationNormalized'].fillna('nan', inplace=True)
test['ContractTime'].fillna('nan', inplace=True)
test['FullDescription'] = test['FullDescription'].str.lower().replace('[^a-zA-Z0-9]', ' ', regex = True)

Xtest1= enc.transform(test[['LocationNormalized', 'ContractTime']].to_dict('records'))
Xtest2= vectorizer.transform(test['FullDescription'])
Xtest=hstack([Xtest1,Xtest2])
ret= clf.predict(Xtest)

from lib.helpersmd import write_to_file
report="{:0.2f} {:0.2f}".format(ret[0],ret[1])
write_to_file(report,"week%d-A%dT%d.out"%(4,1,1))